In [1]:
import numpy as np
import torch
import sys
sys.path.append('../')
from voting_games.werewolf_env_v0 import friendly_env, Roles, Phase
from notebooks.learning_agents.models import ActorCriticAgent
from notebooks.learning_agents.utils import play_static_game, play_recurrent_game
from notebooks.learning_agents.static_agents import (
    random_plurality_villager, 
    random_coordinated_plurality_villager, 
    random_agent,
    random_plurality_wolf,
    revenge_plurality_wolf,
    coordinated_revenge_plurality_wolf)
from pettingzoo.utils.env import ParallelEnv
from pettingzoo.utils.conversions import parallel_to_aec, aec_to_parallel

from tianshou.data import Collector
from tianshou.env import DummyVectorEnv, PettingZooEnv
from tianshou.policy import RandomPolicy, MultiAgentPolicyManager



/root/.local/lib/python3.10/site-packages/gym/envs/registration.py:250: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for plugin in metadata.entry_points().get(entry_point, []):


## Tianshou RL

Tianshou is an RL platform that has seen widespread usage, and is one of the adverstised frameworks on the Farama Foundation.

In [2]:
env = parallel_to_aec(friendly_env(num_agents=10, werewolves=2))
env = PettingZooEnv(env)

# each agent needs a policy
policies = MultiAgentPolicyManager([RandomPolicy() for _ in range(10)], env)

# env needs to be converted to vector format
env = DummyVectorEnv([lambda: env])

#Construct the Collector, which interfaces the policies with the vectorised environment
collector = Collector(policies, env)

result = collector.collect(n_episode=1)

/usr/local/lib/python3.10/site-packages/pettingzoo/utils/conversions.py:232: UserWarning: The base environment `werewolf_plurality_v1` does not have a `render_mode` defined.
  warnings.warn(
